# 🤖 Stock LLM Input Generator

This notebook collects stock information and 5 comprehensive news articles (500+ words each) and formats them into a single string for LLM analysis.

## Features:
- **📊 Stock Information** - Company details, current price, market cap, financial metrics
- **📰 News Articles** - 5 substantial news articles with 500+ words each
- **🔗 Formatted Output** - Combined string ready for LLM input
- **💾 Export Option** - Save the formatted string to a text file

## Usage:
1. Run all cells in order
2. Enter your desired stock symbol when prompted
3. Get a formatted string ready for LLM analysis

---

In [1]:
# Import Required Libraries
import sys
import os
import warnings
import json
from datetime import datetime

warnings.filterwarnings('ignore')

# Import our custom analyzer classes
from stock_data_analyzer import StockAnalyzer
from comprehensive_news_analyzer import ComprehensiveNewsAnalyzer

# Standard libraries
import pandas as pd
from IPython.display import display, HTML, Markdown

print("✅ All libraries imported successfully!")
print("🤖 Stock LLM Input Generator ready!")
print("📊 Preparing to generate formatted stock analysis for LLM...")

# Configure pandas display
pd.set_option('display.max_colwidth', 100)
print("\n🚀 Ready to process stock data for LLM analysis!")

✅ All libraries imported successfully!
🤖 Stock LLM Input Generator ready!
📊 Preparing to generate formatted stock analysis for LLM...

🚀 Ready to process stock data for LLM analysis!


## 📝 Stock Symbol Input

Enter the stock symbol you want to analyze and prepare for LLM input:

Popular stocks:
- **AAPL** - Apple Inc.
- **TSLA** - Tesla Inc.
- **MSFT** - Microsoft Corporation
- **GOOGL** - Alphabet Inc.
- **AMZN** - Amazon.com Inc.
- **NVDA** - NVIDIA Corporation
- **META** - Meta Platforms Inc.

In [67]:
# Stock Symbol Input
STOCK_SYMBOL = "^NDX"  # Change this to any stock symbol you want to analyze


stock_analyzer = StockAnalyzer(STOCK_SYMBOL)
news_analyzer = ComprehensiveNewsAnalyzer(STOCK_SYMBOL)


## 📊 Fetch Basic Stock Information

Collect comprehensive stock information including company details, financials, and recent performance metrics.

## 📰 Fetch and Select 5 News Articles

Retrieve recent news articles and select 5 articles with substantial content (500+ words each) for comprehensive analysis.

In [68]:
from IPython.display import display

# Get basic stock information
stock_info = stock_analyzer.get_stock_info()

# Get recent price statistics (6 months)
price_stats = stock_analyzer.get_price_statistics(period="6mo")

# Get recent trading data (5 days)
recent_data = stock_analyzer.get_historical_data(period="5d")

# Get comprehensive news data
ultra_news = stock_analyzer.get_ultra_comprehensive_news(limit=30)

selected_articles = []

if ultra_news.get('status') == 'success':
    print(f"✅ Found {ultra_news['total_processed_articles']} news articles")
    
    # Filter articles with substantial content (500+ words)
    articles_with_content = [
        article for article in ultra_news['articles'] 
        if not article.get('error') and 
        article.get('full_content', '').strip()
    ]
    
    # Sort by word count (descending) and select top 5
    articles_with_content.sort(key=lambda x: x.get('word_count', 0), reverse=True)
    selected_articles = articles_with_content[:5]
        
else:
    print(f"❌ Failed to collect news articles: {ultra_news.get('error', 'Unknown error')}")

# Create Formatted String for LLM Analysis
print(f"🔗 FORMATTING DATA FOR LLM INPUT - {STOCK_SYMBOL}")
print("=" * 60)

# Start building the formatted string
llm_input_string = []

# Add header
llm_input_string.append(f"STOCK ANALYSIS DATA FOR {STOCK_SYMBOL}")
llm_input_string.append("=" * 80)
llm_input_string.append(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
llm_input_string.append("")

# Add stock information section
llm_input_string.append("COMPANY INFORMATION:")
llm_input_string.append("-" * 40)

if stock_info:
    for key, value in stock_info.items():
        llm_input_string.append(f"{key}: {value}")
else:
    llm_input_string.append("Stock information not available")

llm_input_string.append("")

# Add price statistics section
llm_input_string.append("PRICE ANALYSIS (6 MONTHS):")
llm_input_string.append("-" * 40)

if price_stats:
    llm_input_string.append(f"Current Price: ${price_stats.get('current_price', 'N/A')}")
    llm_input_string.append(f"Period High: ${price_stats.get('period_high', 'N/A')}")
    llm_input_string.append(f"Period Low: ${price_stats.get('period_low', 'N/A')}")
    
    if price_stats.get('change_pct'):
        llm_input_string.append(f"Price Change: {price_stats['change_pct']:.2f}%")
    
    if price_stats.get('volatility'):
        llm_input_string.append(f"Volatility: {price_stats['volatility']:.2f}%")
else:
    llm_input_string.append("Price statistics not available")

llm_input_string.append("")

# Add recent trading data
llm_input_string.append("RECENT TRADING DATA:")
llm_input_string.append("-" * 40)

if recent_data is not None and not recent_data.empty:
    latest_close = recent_data['Close'].iloc[-1]
    daily_change = recent_data['Close'].pct_change().iloc[-1] * 100
    avg_volume = recent_data['Volume'].mean()
    
    llm_input_string.append(f"Latest Close: ${latest_close:.2f}")
    llm_input_string.append(f"Daily Change: {daily_change:+.2f}%")
    llm_input_string.append(f"Average Volume (5 days): {avg_volume:,.0f}")
else:
    llm_input_string.append("Recent trading data not available")

llm_input_string.append("")
llm_input_string.append("")

# Add news articles section
llm_input_string.append("NEWS ARTICLES ANALYSIS:")
llm_input_string.append("=" * 80)

if selected_articles:
    for i, article in enumerate(selected_articles, 1):
        llm_input_string.append(f"ARTICLE {i}:")
        llm_input_string.append("-" * 40)
        llm_input_string.append(f"Title: {article['title']}")
        llm_input_string.append(f"Publisher: {article['publisher']}")
        llm_input_string.append(f"Published: {article['published_date']}")
        llm_input_string.append(f"Word Count: {article.get('word_count', 0):,}")
        llm_input_string.append(f"URL: {article['link']}")
        llm_input_string.append("")
        llm_input_string.append("Content:")
        llm_input_string.append(article.get('full_content', 'Content not available'))
        llm_input_string.append("")
        llm_input_string.append("-" * 80)
        llm_input_string.append("")
else:
    llm_input_string.append("No substantial news articles available")

# Join all parts into final string
final_llm_input = "\n".join(llm_input_string)

print(final_llm_input)

🔍 Processing 10 available news articles for ^NDX...
✅ Found 10 news articles
🔗 FORMATTING DATA FOR LLM INPUT - ^NDX
STOCK ANALYSIS DATA FOR ^NDX
Analysis Date: 2025-06-28 23:09:26

COMPANY INFORMATION:
----------------------------------------
Symbol: ^NDX
Company Name: NASDAQ-100
Current Price: N/A
Market Cap: N/A
PE Ratio: N/A
Dividend Yield: N/A
52 Week High: 22603.22
52 Week Low: 16542.2
Sector: N/A
Industry: N/A

PRICE ANALYSIS (6 MONTHS):
----------------------------------------
Current Price: $22534.19921875
Period High: $N/A
Period Low: $N/A
Price Change: 6.31%
Volatility: 1.91%

RECENT TRADING DATA:
----------------------------------------
Latest Close: $22534.20
Daily Change: +0.39%
Average Volume (5 days): 9,001,732,000


NEWS ARTICLES ANALYSIS:
ARTICLE 1:
----------------------------------------
Title: Small-Cap Stocks Point to a Tech Bubble, Analyst Says
Publisher: Barrons.com
Published: 2025-06-27T11:55:17Z
Word Count: 103
URL: https://www.barrons.com/livecoverage/stock-ma

In [73]:
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv(override=True)

#openai = OpenAI()
openai = OpenAI(base_url="http://localhost:11434/v1",api_key="ollama")
# MODEL = "qwen3:14b"
# MODEL = "gemma3:12b"
MODEL = "llama3.1:8b"
# MODEL = "qwen3:8b"

In [75]:
from IPython.display import display, Markdown, update_display

messages = [
    {
        "role":"system", "content":"You are a stock analyst. With the given information, you would analyse whether to buy or sell a share. You provide reasoning. Quote final desicision like DECISION:#BUY# or DECISION:#SELL# with exact same format. Hash"
    },
    {
        "role":"user","content":f"Analyze the stock with below content and give final decision \n {llm_input_string}"
    }
]

stream = openai.chat.completions.create(messages=messages,model=MODEL,stream=True)
response = ""
display_handle = display(Markdown(""),display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)
        

After analyzing the stock data for ^NDX, I have identified several key points that inform my decision:

* The Nasdaq 100 index has hit a record high this week, indicating strong investor demand.
* However, article 1 suggests that a tech bubble may be building, as the divergence between the Nasdaq 100 and small-cap Russell 2000 is sharper than it was at the height of the dot-com bubble. This raises concerns about potential market volatility.
* Additionally, articles 2, 3, and 4 suggest that the rally is driven by hopes of trade deals, which may not be sustainable in the long term.

Considering these factors, I recommend SELLING ^NDX shares due to the risk of a tech bubble and uncertainty surrounding the sustainability of the current market rally. The volatility in the stock price also increases the risk of significant losses if the market conditions change.

DECISION: #SELL#